In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

# Define hyperparameters
num_classes = 10
input_shape = (224, 224, 3)
batch_size = 32
epochs = 10
learning_rate = 0.0001

# Data Preparation
train_data_dir = "data/train"
validation_data_dir = "data/validation"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# Model creation and fine-tuning
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the pre-trained layers

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Model compilation
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Unfreeze some layers for further fine-tuning
base_model.trainable = True
fine_tune_at = 100  # Fine-tune starting from this layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Lower the learning rate for fine-tuning
optimizer_fine = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer_fine, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Model evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation loss: {loss:.2f}")
print(f"Validation accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save("fine_tuned_image_classification_model.h5")